# Ticks Numpy Analysis Playbook

This notebook is a step-by-step conversion of `ticks_numpy_v2.py` for interactive exploration and visualization of tick data using numpy and pandas.

In [ ]:
# Globals and Imports
from datetime import timezone, datetime, timedelta
import time
import numpy as np
import pandas as pd

gSym = 'EURUSD'
gTimezoneUTC = timezone.utc
gTdOffset = timedelta(hours=2)  # TODO: handle summer/winter time
gTicksPerPeriod = 100
gTimeDelta = timedelta(minutes=240)
gDtTo = datetime(2025, 1, 3, 17, 0, 1, 0, tzinfo=gTimezoneUTC)
gAccount = 'RF5D03'
gDtFrom = gDtTo - gTimeDelta
print("\nFrom:", gDtFrom, "To:", gDtTo)
gDtTo_epoch_ms = int(gDtTo.timestamp() * 1000)
print("\ngDtTo.timestamp()", gDtTo.timestamp(), "gDtTo_epoch_ms:", gDtTo_epoch_ms)

In [ ]:
# Import algotrader and fetch tick data
import algotrader as at
from algotrader._utils import _sprintf

gH = at.Algotrader(gAccount)
gH.mt5_init()

_start = time.time()
_npa = gH.mt5.copy_ticks_range(gSym, gDtFrom, gDtTo, gH.mt5.COPY_TICKS_ALL)
_deltams = int((time.time() - _start) * 1000)

print("\nlen(_npa)", len(_npa), "deltams(_npa):", _deltams, "\n", _npa)
if -1 == gTicksPerPeriod:
    gTicksPerPeriod = len(_npa)
if gTicksPerPeriod > len(_npa):
    print(" try another time when there are more ticks ")

In [ ]:
# Data handling and utilities
def np_array_shift(xs, n):
    e = np.empty_like(xs)
    if n >= 0:
        e[n:] = xs[:-n]
    else:
        e[:n] = xs[-n:]
    return e

_npa = _npa[(len(_npa)-gTicksPerPeriod):]
_npa = np.flip(_npa)

dtype = _npa.dtype
_names = []
_formats = []
for n in _npa.dtype.names:
    _names.append(n)
    _formats.append(_npa.dtype[n])
_names.append('price')
_formats.append(np.dtype(np.float64))
_names.append('spread')
_formats.append(np.dtype(np.int64))
_names.append('tdmsc')
_formats.append(np.dtype(np.int64))
_names.append('np_time_msc')
_formats.append(np.dtype('<S25'))
dtype = np.dtype({'names':_names, 'formats':_formats})
gNpa = np.zeros(gTicksPerPeriod, dtype=dtype)
for n in _npa.dtype.names:
    gNpa[n] = _npa[n]
gNpa['price']  = (gNpa['ask'] + gNpa['bid']) / 2
gNpa['spread'] = (gNpa['ask'] - gNpa['bid']) / 0.00001
gNpa['tdmsc']  = (gNpa['time_msc'] - np_array_shift(gNpa['time_msc'], -1))
gNpa['tdmsc'][-1] = 1000
gNpa['np_time_msc']  = _npa['time_msc'].astype('datetime64[ms]')

In [ ]:
# Data validation and summary statistics
for n in gNpa.dtype.names:
    if 'np_time_msc' != n:
        if np.isnan(np.sum(gNpa[n])):
            strerror = _sprintf("NAN ERROR sym[%s] within column[%s] %s", gSym, n, str(gNpa[n]))
            raise ValueError(strerror)

gTimeLastTickMS = gDtTo_epoch_ms - gNpa['time_msc'][0]
print("\nlen(gNpa)", len(gNpa), "gTimeLastTickMS:", gTimeLastTickMS, "\n", gNpa)

tnow = np.array(gDtTo_epoch_ms).astype('datetime64[ms]')
topen = gNpa['time_msc'][0].astype('datetime64[ms]')
tclose = gNpa['time_msc'][-1].astype('datetime64[ms]')
print(gSym, "TIME", gTimeLastTickMS, "ms", " t:", tnow, " t0:", topen, " tn:", tclose)

In [ ]:
# Price and spread analysis
oc = int(round((gNpa['price'][0] - gNpa['price'][-1]) / 0.00001, 0))
print(gSym, "PRICE  OC:", oc, "open:", gNpa['price'][0], "@", topen, "close:", gNpa['price'][-1], "@", tclose)

idxmax = gNpa['price'].argmax(axis=0)
idxmin = gNpa['price'].argmin(axis=0)
thigh = gNpa['time_msc'][idxmax].astype('datetime64[ms]')
tlow = gNpa['time_msc'][idxmin].astype('datetime64[ms]')
hl = int(round((gNpa['price'][idxmax] - gNpa['price'][idxmin]) / 0.00001, 0))
print(gSym, "PRICE  HL:", hl, "high:", gNpa['price'][idxmax], "@", thigh, "low:", gNpa['price'][idxmin], "@", tlow)

idxmax = gNpa['spread'].argmax(axis=0)
idxmin = gNpa['spread'].argmin(axis=0)
spread_thigh = gNpa['time_msc'][idxmax].astype('datetime64[ms]')
spread_tlow = gNpa['time_msc'][idxmin].astype('datetime64[ms]')
spread_hl = gNpa['spread'][idxmax] - gNpa['spread'][idxmin]
print(gSym, "SPREAD HL:", spread_hl, "high:", gNpa['spread'][idxmax], "@", spread_thigh, "low:", gNpa['spread'][idxmin], "@", spread_tlow)

In [ ]:
# Visualize price track
import matplotlib.pyplot as plt
gNpa = np.flip(gNpa)
gNpaPrice0 = gNpa['price'][0]
gNPAoffset = gNpaPrice0 * np.ones(len(gNpa))
gNpaRealTrack = (gNpa['price'] - gNPAoffset) / 0.00001

myarray = gNpaRealTrack
myarray = np.round(myarray)
myarray = myarray.astype(int)
pcmax = myarray.max() - myarray.min()
x = np.arange(len(gNpa))
y = myarray
pcm = 0
if 0 != pcmax:
    pcm = round((np.polyfit(x, y, 1)[0] * len(gNpa)) / pcmax, 1)
pcmreal = float("%.1f" % pcm)
pcmrealraw = np.polyfit(x, y, 1)[0]
pcmaxreal = int(pcmax)

plt.figure()
plt.plot(x, gNpaRealTrack, label='price', color='r', linewidth=0.5)
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.show()